In [1]:
import pandas as pd

In [2]:
df1 = pd.read_pickle('2013-2023_plays.pkl')

In [3]:
aaa = ['down','goal_to_go','ydstogo','fourth_down_converted','fourth_down_failed','series_result','field_goal_attempt','punt_attempt','spread_line','posteam_type'

]
df= df1[aaa]

In [4]:
df = df[df['down']==4]

In [5]:
mask = (
    (df['fourth_down_converted'] == 0) &
    (df['fourth_down_failed'] == 0) &
    (df['field_goal_attempt'] == 0) &
    (df['punt_attempt'] == 0)
)

# Use the mask to filter the DataFrame
rows_to_drop = df[mask]
df = df[~mask]

mask2 = (df['punt_attempt'] == 1) | (df['field_goal_attempt'] == 1)
df = df[~mask2]


def adjust_spread(row):
    if row['posteam_type'] == 'away':
        return -row['spread_line']
    else:
        return row['spread_line']
    

# Apply the function across the DataFrame rows
df['adjusted_spread_line'] = df.apply(adjust_spread, axis=1)

df['posteam_is_home'] = (df['posteam_type'] == 'home').astype(int)
df.drop('posteam_type',axis=1,inplace=True)

In [6]:
df = df[['goal_to_go','fourth_down_converted','ydstogo','adjusted_spread_line','posteam_is_home']].dropna()

df = df[['fourth_down_converted','ydstogo']]

In [7]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Assuming df is predefined and includes the necessary columns

# Define features and target
X = df.drop('fourth_down_converted',axis=1)
y = df['fourth_down_converted']    # target

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train logistic regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict on the testing set
y_pred = log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Model Accuracy: 0.64
Confusion Matrix:
[[457 524]
 [207 856]]


In [8]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix


In [9]:
# Create a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=1000, random_state=27)

# Train the model
rf_model.fit(X_train, y_train)
# Predict on the testing set
y_pred = rf_model.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.64
Confusion Matrix:
[[623 358]
 [371 692]]


In [12]:
import pickle
with open('GFI_prediction_model.pkl','wb') as file:
    pickle.dump(rf_model,file)

In [10]:
import tensorflow as tf

# Build the ANN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'binary_crossentropy' for binary classification
              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


# Train the model again
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")


Epoch 1/50
  1/120 [..............................] - ETA: 19s - loss: 0.7025 - accuracy: 0.5312

2024-04-26 13:36:00.925984: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


120/120 [==============================] - 0s 1ms/step - loss: 0.6631 - accuracy: 0.6049 - val_loss: 0.6513 - val_accuracy: 0.6090
Epoch 2/50
120/120 [==============================] - 0s 483us/step - loss: 0.6508 - accuracy: 0.6159 - val_loss: 0.6480 - val_accuracy: 0.6195
Epoch 3/50
120/120 [==============================] - 0s 478us/step - loss: 0.6509 - accuracy: 0.6211 - val_loss: 0.6477 - val_accuracy: 0.6090
Epoch 4/50
120/120 [==============================] - 0s 468us/step - loss: 0.6486 - accuracy: 0.6253 - val_loss: 0.6465 - val_accuracy: 0.6195
Epoch 5/50
120/120 [==============================] - 0s 472us/step - loss: 0.6506 - accuracy: 0.6148 - val_loss: 0.6571 - val_accuracy: 0.6038
Epoch 6/50
120/120 [==============================] - 0s 1ms/step - loss: 0.6493 - accuracy: 0.6224 - val_loss: 0.6459 - val_accuracy: 0.6195
Epoch 7/50
120/120 [==============================] - 0s 547us/step - loss: 0.6467 - accuracy: 0.6172 - val_loss: 0.6496 - val_accuracy: 0.6090
Epoch 8